Connecting google drive to access geodata and connecting to Github to keep track of my changes for this program

In [100]:
# mounting Google Drive and connecting to my GitHub repository
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Read the token from a file stored in Drive
with open('/content/drive/MyDrive/to/USDA-CostaRica-GeoAnalysis.txt') as f:
    token = f.read().strip()

# Clone the repository (corrected URL with token)
repo_url = f"https://{token}@github.com/jacquelinesanchezA1/USDA-Forest.git"
# !git clone https://github.com/jacquelinesanchezA1/USDA-Forest.git

# Only clone if the repo doesn't already exist
if not os.path.exists('/content/USDA-Forest'):
    !git clone {repo_url}
else:
    # If the repository exists, pull the latest changes to sync with remote
    %cd /content/USDA-Forest/a.Main-USDA-Project-Group-Version/
    !git pull origin main

# Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/USDA-Forest/a.Main-USDA-Project-Group-Version
From https://github.com/jacquelinesanchezA1/USDA-Forest
 * branch            main       -> FETCH_HEAD
Already up to date.


Connecting to GEE API <br>
why are we connecting to GEE API? Connecting to GEE will allow us access to satellite imagery (Landsat, Sentinel), climate data, topography, land cover, and more

In [101]:
import ee, geemap
# trigger the authentication flow.
ee.Authenticate()

# initialize the library.
ee.Initialize(project='ee-jacquelinesancheza14') #you will want to select your personal cloud project

## Importing necessary libraries

In [102]:
#import packages
import geopandas as gpd, pandas as pd, os, numpy as np
import ee, geemap

## Uploading the GeoDataframe from the .shp file saved in Drive

In [103]:
# Load the shapefile (update the path as necessary)
gdf = gpd.read_file('/content/drive/My Drive/Resources/Clasification_Plots.shp')

# Display the first few rows
gdf.head()

# Print the column names
# print(gdf.columns)

,Source.Nam,plotid,sampleid,lon,lat,sample_geo,Uso,Cobertura,Vegetacion,Herbaceas,Pasto_Arb,Cultivo,Humedal,Terreno,Agua,Otra_clase,SAF,Cambios15_,Gana_Perdi,geometry
0,ceo-ACAHN-puntos-Mapa-de-tipos-de-Bosque-y-otr...,2900,11597,-84.908874,10.874762,POINT(-84.90887419133645 10.874761552334505),Bosque,Vegetacion,Arboles,None,None,None,None,None,None,None,None,No se determina,None,POINT (-84.90887 10.87476)
1,ceo-ACAHN-puntos-Mapa-de-tipos-de-Bosque-y-otr...,2900,11598,-84.908874,10.875185,POINT(-84.90887419133645 10.875185000000002),Pastos,Vegetacion,Herbaceas,Gramineas,Pastos mezclados (70-90%),None,None,None,None,None,None,No se determina,None,POINT (-84.90887 10.87519)
2,ceo-ACAHN-puntos-Mapa-de-tipos-de-Bosque-y-otr...,2900,11599,-84.908874,10.875608,POINT(-84.90887419133645 10.875608447064252),Pastos,Vegetacion,Herbaceas,Gramineas,Pastos mezclados (70-90%),None,None,None,None,None,None,No se determina,None,POINT (-84.90887 10.87561)
3,ceo-ACAHN-puntos-Mapa-de-tipos-de-Bosque-y-otr...,2900,11600,-84.908443,10.874762,POINT(-84.908443 10.874761552334505),Bosque,Vegetacion,Arboles,None,None,None,None,None,None,None,None,No se determina,None,POINT (-84.90844 10.87476)
4,ceo-ACAHN-puntos-Mapa-de-tipos-de-Bosque-y-otr...,2900,11601,-84.908443,10.875185,POINT(-84.908443 10.875185000000002),Bosque,Vegetacion,Arboles,None,None,None,None,None,None,None,None,No se determina,None,POINT (-84.90844 10.87519)


## Subset the geodataframe to the columns we are interested in and display the dataframe

In [94]:
k_clms = ['plotid','sampleid','Vegetacion','Herbaceas', 'Pasto_Arb', 'Cultivo','Humedal', 'Terreno','Agua','Otra_clase','SAF','Cambios15_','Gana_Perdi','geometry']
gdf_s=gdf[k_clms]
gdf_s

,plotid,sampleid,Vegetacion,Herbaceas,Pasto_Arb,Cultivo,Humedal,Terreno,Agua,Otra_clase,SAF,Cambios15_,Gana_Perdi,geometry
0,2900,11597,Arboles,None,None,None,None,None,None,None,None,No se determina,None,POINT (-84.90887 10.87476)
1,2900,11598,Herbaceas,Gramineas,Pastos mezclados (70-90%),None,None,None,None,None,None,No se determina,None,POINT (-84.90887 10.87519)
2,2900,11599,Herbaceas,Gramineas,Pastos mezclados (70-90%),None,None,None,None,None,None,No se determina,None,POINT (-84.90887 10.87561)
3,2900,11600,Arboles,None,None,None,None,None,None,None,None,No se determina,None,POINT (-84.90844 10.87476)
4,2900,11601,Arboles,None,None,None,None,None,None,None,None,No se determina,None,POINT (-84.90844 10.87519)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101155,904894,3619577,Arboles,None,None,None,None,None,None,None,None,No se determina,None,POINT (-83.84296 10.02921)
101156,904894,3619578,Arboles,None,None,None,None,None,None,None,None,No se determina,None,POINT (-83.84296 10.02964)
101157,904894,3619579,Arboles,None,None,None,None,None,None,None,None,No se determina,None,POINT (-83.84253 10.02879)
101158,904894,3619580,Arboles,None,None,None,None,None,None,None,None,No se determina,None,POINT (-83.84253 10.02921)


## Display summary statistics

In [95]:
# display(gdf_s.describe())
display(gdf.describe(include='object'))

,Source.Nam,sample_geo,Uso,Cobertura,Vegetacion,Herbaceas,Pasto_Arb,Cultivo,Humedal,Terreno,Agua,Otra_clase,SAF,Cambios15_,Gana_Perdi
count,101160,101160,100944,100944,94158,24465,23291,8398,3926,5558,887,5449,8398,100944,3232
unique,10,91917,7,5,7,3,3,10,5,3,2,6,3,3,2
top,ceo-ACAHN-puntos-Mapa-de-tipos-de-Bosque-y-otr...,POINT(-84.90887419133645 10.874761552334505),Bosque,Vegetacion,Arboles,Gramineas,Pastos mezclados (70-90%),Otro,Pantano (Palustre),Otras superficies,Continentales,Suelo desnudo,Cultivo Puro (90-100%),No,Perdida de Bosque
freq,10125,2,58708,94158,61011,20984,12637,2569,1592,2187,818,2721,5217,90237,1774


## Make GeoSeries of the study area and create convex hull

In [96]:
chul=gpd.GeoSeries(gdf_s.unary_union.convex_hull,crs=gdf_s.crs)

<ipython-input-96-08c7e26ad7ca>:1: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  chul=gpd.GeoSeries(gdf_s.unary_union.convex_hull,crs=gdf_s.crs)


## Create definitions for the `median` and `medoid` procedures

As you noticed we are creating a function to mask unwanted pixels eg clouds, shadows etc so that we can have a good view of the land without being disrupted by the clouds. Further down the analysis we are going to analyze the land in a good time period where clouds are less prone in Costa Rica.

In [97]:
# function to mask unwanted pixels (clouds, shadows, etc.) from Landsat 8 SR images

def maskL8sr(image): # Landsat 8 images
    # Bit 0 - Fill
    # Bit 1 - Dilated Cloud
    # Bit 2 - Cirrus
    # Bit 3 - Cloud
    # Bit 4 - Cloud Shadow

    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)

    # Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)

    #Replace the original bands with the scaled ones and apply the masks.
    return image.addBands(opticalBands, overwrite=True).addBands(thermalBands, overwrite=True).updateMask(qaMask).updateMask(saturationMask)



# Function to create a median mosaic of a given image collection
def median_mosaic(image,fltr=None,refl_bands=['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']):
    if(fltr is None):
        inCollection = image.filter(fltr).select(refl_bands)
    else:
        inCollection = image.filter(fltr).select(refl_bands)

    # Return the median composite image
    return inCollection.median()



# Helper function to calculate medoid from an image collection
def _medoid(col):
    median = ee.ImageCollection(col).median()
    diff=ee.Image(col).subtract(median).pow(ee.Image.constant(2))

    # Return the sum of the squared differences, adding the bands back to the collection
    return diff.reduce('sum').addBands(col)



# Function to create a medoid mosaic of a given image collection
def medoid_mosaic(image, fltr,refl_bands=['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']):
    if(fltr is None):
        inCollection = image.filter(fltr).select(refl_bands)
    else:
        inCollection = image.filter(fltr).select(refl_bands)

    # Apply the _medoid function and reduce the image collection to the medoid composite
    medoid = inCollection.map(_medoid)
    medoid = ee.ImageCollection(medoid).reduce(ee.Reducer.min(7)).select([1,2,3,4,5,6], refl_bands)
    return medoid

## Set various variable and create the mdoid surface on ee

Good question to ask perhaps to Andy and John: When does cloud season take place?
Since Costa Rica is in the lower hemisphere than north america.

In [98]:
#make lists fo band names for selections
lc8_bands = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10', 'QA_PIXEL']#landsat band names
tgt_bands = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2', 'TEMP', 'QA_PIXEL']#common band names
refl_bands = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']#bands we care about

#specify start and end dates for the image filter
startDate = '2021-01-01'
endDate = '2024-07-01'

#Specify julian dates for filter. Here we want to select sunny months
julianStart1 = 350# Starting Julian Date (for landsat median cloud free )
julianEnd1 = 365
julianStart2 = 1
julianEnd2 = 150# Ending Julian date (for landsat median cloud free)

#define the study area extent from our convex hull
#geo=geemap.gdf_to_ee(gpd.GeoDataFrame(geometry=chul)) #convert our convex hull into a ee feature class object

#make the ee collection
l8_col=ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')

#set various filters
#f_bnds=ee.Filter.bounds(geometry=geo)
f_date=ee.Filter.date(startDate,endDate)
f_cr1=ee.Filter.calendarRange(julianStart1,julianEnd1)
f_cr2=ee.Filter.calendarRange(julianStart2,julianEnd2)
f_or=ee.Filter.Or(f_cr1,f_cr2)
f_and=ee.Filter.And(f_date,f_or)

#use our filter on the landsat collection
l8=l8_col.filter(f_and).map(maskL8sr)
l8r=l8.select(lc8_bands,tgt_bands)

#call the medoid function
medoid = medoid_mosaic(l8r,fltr=f_and,refl_bands=refl_bands)

## Create a generic Map using geemap and then map the DEM surface

In [104]:
Map = geemap.Map()
# Map.add_basemap('HYBRID')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Add the EE layers to the map

In [ ]:
# get the dem from EE
dem = ee.Image('USGS/SRTMGL1_003')

# add the dem to the map
Map.addLayer(dem, {}, 'SRTM DEM', True, 0.5)

# Set visualizetion parameters for the map
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# add earth engine layer to map
Map.addLayer(medoid, {"bands": ["RED", "GREEN", "BLUE"],
                      'min': -0.2,
                      'max': 0.3},
             'Landsat 8', True, 0.7)


# Add the DEM to the map
Map.addLayer(dem, vis_params, 'SRTM DEM', True, 1) # try 0.5

In [99]:
# move it to the USDA-Forest repository
!cp /content/drive/MyDrive/Colab_Notebooks/USDA_CostaRica_1.ipynb /content/USDA-Forest/a.Main-USDA-Project-Group-Version/

# Navigate to the cloned repository directory
%cd /content/USDA-Forest/a.Main-USDA-Project-Group-Version/

# Add the changes (including the moved notebook)
!git add .

# Commit the changes with a relevant commit message
!git commit -m "Added updated notebook"

# Push the changes to GitHub
!git push origin main

/content/USDA-Forest/a.Main-USDA-Project-Group-Version
[main 8d66a5c] Added updated notebook
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite a.Main-USDA-Project-Group-Version/USDA_CostaRica_1.ipynb (93%)
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 1.89 KiB | 1.89 MiB/s, done.
Total 4 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/jacquelinesanchezA1/USDA-Forest.git
   efda901..8d66a5c  main -> main


Saving changes and uploading/pushing them to gh repo